In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import BingChatAccess
import ChatGPTAccess
import json
import os
from pptx import Presentation

In [4]:
chatGPT = ChatGPTAccess.ChatGPTAccess()

FileNotFoundError: [Errno 2] No such file or directory: '.ChatGPTtoken'

In [5]:
bingChat = BingChatAccess.BingChatAccess()

In [6]:
options = {"len" : 30, "adj" : "historical", "top" : "Seven Years' War", "foc" : "Describe all the theatres of war. Don't describe events separately, but focus on overall progress of the war. Create separate slides for Paris and Hubertsburg peace", "folder" : "E:/honzi/OneDrive/IVP/prezentace", "lang" : "CZECH", "pointsMin" : 4, "pointsMax" : 6, "outlines" : 4, "subtitle" : "Jan Slíva", "saveName" : "Seven Years' War IV"}

In [7]:
with open('prompts.json') as f:
    prompts = json.load(f)

In [8]:
first = prompts["outline"].format(**options)
print(first)

Create outline for exhaustive historical presentation about Seven Years' War. Describe all the theatres of war. Don't describe events separately, but focus on overall progress of the war. Create separate slides for Paris and Hubertsburg peace For each slide write only brief desriptive name. There hast to be at least 30 slides. Format the list as json. The structure of the json has to be

{
"slides":[
<name of slide 1>,
<name of slide 2>,
<name of slide 3>,
<name of slide 4>,
...
<name of slide N>
]
}



In [9]:
answers = []
outlines = []
for i in range(options["outlines"]):
    answers.append(await bingChat.answerPropt(first, 500, newConv=True))

gpt-4 tokens: 130, maximum 4096 - 500
BingChat: Using: gpt-4
BingChat: Sending prompt
No message found
BingChat: The try wasn't succesful, trying again
BingChat: Sending prompt
CaptchaChallenge: User needs to solve CAPTCHA to continue.
BingChat: The try wasn't succesful, trying again


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fa370bfcc10>


BingChat: Sending prompt
gpt-4 tokens: 130, maximum 4096 - 500
BingChat: Using: gpt-4
BingChat: Sending prompt
gpt-4 tokens: 130, maximum 4096 - 500
BingChat: Using: gpt-4
BingChat: Sending prompt
Cannot connect to host sydney.bing.com:443 ssl:<ssl.SSLContext object at 0x7fa371b305f0> [Name or service not known]
BingChat: The try wasn't succesful, trying again
BingChat: Sending prompt
gpt-4 tokens: 130, maximum 4096 - 500
BingChat: Using: gpt-4
BingChat: Sending prompt
No message found
BingChat: The try wasn't succesful, trying again
BingChat: Sending prompt


In [11]:
print(answers)

[{'text': 'I have created an outline for a historical presentation about the Seven Years\' War, based on the information I found on the web. Here is the list of slides in JSON format:\n\n{\n"slides":[\n"Introduction: The Causes and Context of the Seven Years\' War",\n"The European Theatre: The Diplomatic Revolution and the Alliance System",\n"The North American Theatre: The French and Indian War (1754-1763)",\n"The Indian Theatre: The Third Carnatic War (1757-1763)",\n"The Caribbean Theatre: The Naval War in the West Indies (1756-1763)",\n"The African Theatre: The Anglo-French Rivalry in West Africa (1758-1763)",\n"The Asian Theatre: The Anglo-French Conflict in the Philippines (1762-1764)",\n"The European Theatre: The Prussian Invasion of Saxony and the Battle of Lobositz (1756)",\n"The European Theatre: The Austrian Counterattack and the Battle of Kolin (1757)",\n"The European Theatre: The Prussian Victory at Rossbach and Leuthen (1757)",\n"The European Theatre: The Russian Intervent

In [22]:
outlines = []
for i in range(len(answers)):
    topics  = json.loads(answers[i]["text"][(answers[i]["text"].find("{")):(answers[i]["text"].find("}")+1)])
    topics = topics[list(topics.keys())[0]]
    outlines.append(topics)

    outline = []
    for top in topics:
        outline.append("    - " + top)
    outline = "\n".join(outline)
    
    print("Option " + str(i) + ":")
    print(outline, end="\n\n")

Option 0:
    - Introduction: The Causes and Context of the Seven Years' War
    - The European Theatre: The Diplomatic Revolution and the Alliance System
    - The North American Theatre: The French and Indian War (1754-1763)
    - The Indian Theatre: The Third Carnatic War (1757-1763)
    - The Caribbean Theatre: The Naval War in the West Indies (1756-1763)
    - The African Theatre: The Anglo-French Rivalry in West Africa (1758-1763)
    - The Asian Theatre: The Anglo-French Conflict in the Philippines (1762-1764)
    - The European Theatre: The Prussian Invasion of Saxony and the Battle of Lobositz (1756)
    - The European Theatre: The Austrian Counterattack and the Battle of Kolin (1757)
    - The European Theatre: The Prussian Victory at Rossbach and Leuthen (1757)
    - The European Theatre: The Russian Intervention and the Battle of Zorndorf (1758)
    - The European Theatre: The Austrian Resurgence and the Battle of Hochkirch (1758)
    - The European Theatre: The Prussian Di

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [23]:
index = int(input("What outline to use?"))


topics  = outlines[index]

outline = []
for top in topics:
    outline.append("    - " + top)
outline = "\n".join(outline)

print(outline)

    - Introduction: The Causes and Context of the Seven Years' War
    - The European Theatre: The Diplomatic Revolution and the Alliance System
    - The North American Theatre: The French and Indian War (1754-1763)
    - The Indian Theatre: The Third Carnatic War (1757-1763)
    - The Caribbean Theatre: The Naval War in the West Indies (1756-1763)
    - The African Theatre: The Anglo-French Rivalry in West Africa (1758-1763)
    - The Asian Theatre: The Anglo-French Conflict in the Philippines (1762-1764)
    - The European Theatre: The Prussian Invasion of Saxony and the Battle of Lobositz (1756)
    - The European Theatre: The Austrian Counterattack and the Battle of Kolin (1757)
    - The European Theatre: The Prussian Victory at Rossbach and Leuthen (1757)
    - The European Theatre: The Russian Intervention and the Battle of Zorndorf (1758)
    - The European Theatre: The Austrian Resurgence and the Battle of Hochkirch (1758)
    - The European Theatre: The Prussian Disaster at 

In [24]:
with open('prompts.json') as f:
    prompts = json.load(f)

prompt = prompts["first_slide"].format(outline=outline, topOfSlide=topics[0], **options)

slides = []
slides.append(await bingChat.answerPropt(prompt, 1000, newConv=True))
print(slides[0]["text"])

gpt-4 tokens: 926, maximum 4096 - 1000
BingChat: Using: gpt-4
BingChat: Sending prompt
No message found
BingChat: The try wasn't succesful, trying again
BingChat: Sending prompt
## Introduction: The Causes and Context of the Seven Years' War
- The Seven Years' War was a global conflict that lasted from 1756 to 1763 and involved most of the major European powers and their colonies[^1^] [^2^] [^5^].
- The war had two main theatres: the European theatre, where Prussia and Britain fought against Austria, France, Russia, and their allies; and the colonial theatre, where Britain and France competed for supremacy in North America, India, the Caribbean, Africa, and Asia[^1^] [^2^] [^5^].
    - The European theatre was triggered by the diplomatic revolution of 1756, when France and Austria formed an alliance against their traditional rival Prussia, which was supported by Britain[^2^] [^5^].
    - The colonial theatre was sparked by the French and Indian War (1754-1763), which began as a local d

In [25]:
print(slides[0]["text"])

## Introduction: The Causes and Context of the Seven Years' War
- The Seven Years' War was a global conflict that lasted from 1756 to 1763 and involved most of the major European powers and their colonies[^1^] [^2^] [^5^].
- The war had two main theatres: the European theatre, where Prussia and Britain fought against Austria, France, Russia, and their allies; and the colonial theatre, where Britain and France competed for supremacy in North America, India, the Caribbean, Africa, and Asia[^1^] [^2^] [^5^].
    - The European theatre was triggered by the diplomatic revolution of 1756, when France and Austria formed an alliance against their traditional rival Prussia, which was supported by Britain[^2^] [^5^].
    - The colonial theatre was sparked by the French and Indian War (1754-1763), which began as a local dispute over the Ohio River Valley between British and French colonists and their Native American allies[^1^] [^5^].
- The war was marked by several decisive battles, sieges, and 

In [64]:
for i in range(1, len(topics)):
    gen = [slides[0]["text"]]
    for j in range(max(1, i-3), i-1):
        gen.append(slides[j]["text"])
    gen = "\n\n".join(gen)
    prompt = prompts["other_slide"].format(outline=outline, topOfSlide=topics[i], createdSlides=gen, **options)
    slides.append(await bingChat.answerPropt(prompt, 1000, newConv=True))
    print(slides[i]["text"])


gpt-4 tokens: 1234, maximum 4096 - 1000
BingChat: Using: gpt-4
BingChat: Sending prompt
## Europe: The Diplomatic Revolution and the alliance system
- The Diplomatic Revolution of 1756 was the reversal of longstanding alliances in Europe between the War of the Austrian Succession and the Seven Years' War[^1^] [^2^]
- Austria went from an ally of Britain to an ally of France, the Dutch Republic, a long-standing British ally, became more anti-British and took a neutral stance, while Prussia became an ally of Britain[^1^] [^2^]
    - The most influential diplomat involved was an Austrian statesman, Wenzel Anton von Kaunitz[^1^]
    - The change was part of the stately quadrille, a constantly shifting pattern of alliances throughout the 18th century in efforts to preserve or upset the European balance of power[^1^]
- The diplomatic change was triggered by a separation of interests among Austria, Britain, and France after the Peace of Aix-la-Chapelle (1748), which ended the War of the Austr

In [ ]:
import deepl

with open(".DeepLtoken") as f:
    auth_key = f.read()
translator = deepl.Translator(auth_key)

In [ ]:
import re

def formatSlide(text):
    lines = text.splitlines()
    ret = []
    if lines[0][0] != "#":
        del lines[0]
    for line in lines:
        if (line != "") and (line[0] != "["):
            line = re.sub("\*\*", "", line)
            line = re.sub(" ?\[.*?\] ?", "", line)
            ret.append(line)
    ret = "\n".join(ret)
    return ret

def transate(text, lang="CS"):
    if text == "":
        return ""
    return translator.translate_text(text, target_lang=lang).text

In [ ]:

nameOfPres = transate(options["saveName"])

In [ ]:
pres_list = list(map(lambda x: transate(formatSlide(x["text"])), slides))
pres = "\n\n".join(pres_list)

with open(os.path.join(options["folder"], nameOfPres + ".md"), "w", encoding="UTF-8") as f:
    f.write(pres)

In [ ]:
from pptx.enum.lang import MSO_LANGUAGE_ID
from pptx.util import Pt

def addSlide(power, text, language, font_size=20):
    lines = text.splitlines()
    if len(lines) == 0:
        return

    lang_id = getattr(MSO_LANGUAGE_ID, language)

    lay = power.slide_layouts[1]
    slide = power.slides.add_slide(lay)

    slide.shapes.title.text = re.sub("^## +", "", lines[0])
    slide.shapes.title.text_frame.paragraphs[0].font.language_id = lang_id

    text_frame = slide.shapes.placeholders[1].text_frame

    first = True
    for line in lines[1:]:
        if first:
            p = text_frame.paragraphs[0]
            first = False
        else:
            p = text_frame.add_paragraph()
        p.text = re.sub("^ *- +", "", line)
        p.font.language_id = lang_id
        p.font.size = Pt(font_size)
        if line[0] == "-":
            p.level = 0
        else:
            p.level = 1

    

power = Presentation("template.pptx")
power.core_properties.language = options["lang"]

lay = power.slide_layouts[0]
first = power.slides.add_slide(lay)
first.shapes.title.text = transate(options["top"])
first.placeholders[1].text= options["subtitle"]

for slide in pres_list:
    addSlide(power, slide, options["lang"])
power.save(os.path.join(options["folder"], nameOfPres + ".pptx"))

In [ ]:
with open(os.path.join(options["folder"], nameOfPres + " sources.txt"), "w", encoding="UTF-8") as f:
    for i in range(len(slides)):
        if pres_list[i].rstrip() == "":
            continue
        f.write(pres_list[i].splitlines()[0] + "\n")
        for source in slides[i]["sources"]:
            f.write(source + "\n")
        f.write("\n")